In [4]:
%pip install nfl_data_py
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [44]:
import nfl_data_py as nfl
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.ticker as plticker
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [74]:
years = [2022, 2023]
nfl.import_weekly_data(years=years, downcast= True)

Downcasting floats.


,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,opponent_team
0,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,1,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,10.38,10.38,NaN
1,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,2,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,9.40,9.40,NaN
2,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,3,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,14.74,14.74,NaN
3,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,4,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,25.40,25.40,NaN
4,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,5,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,19.74,19.74,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5400,00-0039165,Z.Charbonnet,Zach Charbonnet,RB,RB,https://static.www.nfl.com/image/private/f_aut...,SEA,2023,14,REG,...,-0.787724,0,-0.666667,0.032258,-0.025532,0.030515,0.0,4.80,5.80,SF
5401,00-0039165,Z.Charbonnet,Zach Charbonnet,RB,RB,https://static.www.nfl.com/image/private/f_aut...,SEA,2023,15,REG,...,-0.442067,0,0.000000,0.031250,0.012658,0.055736,0.0,1.60,1.60,PHI
5402,00-0039165,Z.Charbonnet,Zach Charbonnet,RB,RB,https://static.www.nfl.com/image/private/f_aut...,SEA,2023,16,REG,...,-1.961893,0,0.307692,0.055556,-0.056034,0.044109,0.0,-0.40,0.60,TEN
5403,00-0039165,Z.Charbonnet,Zach Charbonnet,RB,RB,https://static.www.nfl.com/image/private/f_aut...,SEA,2023,17,REG,...,1.644468,0,3.000000,0.151515,0.057778,0.267717,0.0,4.10,9.10,PIT


In [75]:
df = nfl.import_weekly_data(years=years, downcast= True)
df.to_csv('nfl_data.csv', index=False)

Downcasting floats.


In [59]:
top_5_target_share = nfl.import_weekly_data(years=years, downcast= True).sort_values('target_share', ascending=False).head(5)[['player_display_name', 'week', 'target_share']]
print(top_5_target_share)

Downcasting floats.
     player_display_name  week  target_share
466         Stefon Diggs     6      0.551724
2409      Terry McLaurin     6      0.523810
358        Davante Adams    10      0.520000
2882    Justin Jefferson     4      0.500000
2444          A.J. Brown     7      0.500000


In [60]:
top_5_fantasy_ppr = nfl.import_weekly_data(years=years, downcast=True).sort_values('fantasy_points_ppr', ascending=False).head(5)[['player_display_name', 'week', 'fantasy_points_ppr']]
print(top_5_fantasy_ppr)

Downcasting floats.
      player_display_name  week  fantasy_points_ppr
3423        Ja'Marr Chase     5           52.200001
422          Amari Cooper    16           51.500000
5160         Devon Achane     3           51.299999
1801           D.J. Moore     5           49.000000
881   Christian McCaffrey     4           48.700001


In [46]:
qb_columns = ['passer_player_name', 'pass_attempt', 'complete_pass', 'pass_touchdown', 'yards_gained', 'interception']
rb_columns = ['rusher_player_name', 'rush_attempt', 'rush_touchdown', 'yards_gained']
wr_columns = ['receiver_player_name', 'complete_pass', 'pass_touchdown', 'yards_gained']

qb_data = nfl.import_pbp_data(years=[2023], columns=qb_columns)
rb_data = nfl.import_pbp_data(years=[2023], columns=rb_columns)
wr_data = nfl.import_pbp_data(years=[2023], columns=wr_columns)

2023 done.
Downcasting floats.
2023 done.
Downcasting floats.
2023 done.
Downcasting floats.


In [47]:
# Calculate quarterback stats
qb_stats = qb_data.groupby('passer_player_name').agg({
    'pass_attempt': 'sum',
    'complete_pass': 'sum',
    'pass_touchdown': 'sum',
    'yards_gained': 'sum',
    'interception': 'sum'
})

# Calculate running back stats
rb_stats = rb_data.groupby('rusher_player_name').agg({
    'rush_attempt': 'sum',
    'rush_touchdown': 'sum',
    'yards_gained': 'sum'
})

# Calculate wide receiver stats
wr_stats = wr_data.groupby('receiver_player_name').agg({
    'complete_pass': 'sum',
    'pass_touchdown': 'sum',
    'yards_gained': 'sum'
})

In [48]:
# Rank quarterbacks
qb_stats['fantasy_points'] = qb_stats['complete_pass'] * 0.4 + qb_stats['pass_touchdown'] * 4 - qb_stats['interception'] * 2
top_qbs = qb_stats.sort_values(by='fantasy_points', ascending=False).head(10)

# Rank running backs
rb_stats['fantasy_points'] = rb_stats['rush_touchdown'] * 6 + rb_stats['yards_gained'] * 0.1
top_rbs = rb_stats.sort_values(by='fantasy_points', ascending=False).head(10)

# Rank wide receivers
wr_stats['fantasy_points'] = wr_stats['pass_touchdown'] * 6 + wr_stats['yards_gained'] * 0.1
top_wrs = wr_stats.sort_values(by='fantasy_points', ascending=False).head(10)

In [51]:
# Display the top quarterbacks
print("Top 5 Quarterbacks:")
print(top_qbs.head(5).index)

# Display the top 5 running backs
print("\nTop 5 Running Backs:")
print(top_rbs.head(5).index)

# Display the top 5 wide receivers
print("\nTop 5 Wide Receivers:")
print(top_wrs.head(5).index)

Top 5 Quarterbacks:
Index(['D.Prescott', 'J.Goff', 'J.Love', 'T.Tagovailoa', 'P.Mahomes'], dtype='object', name='passer_player_name')

Top 5 Running Backs:
Index(['C.McCaffrey', 'B.Robinson', 'R.Mostert', 'D.Henry', 'K.Williams'], dtype='object', name='rusher_player_name')

Top 5 Wide Receivers:
Index(['T.Hill', 'C.Lamb', 'A.St. Brown', 'M.Evans', 'D.Moore'], dtype='object', name='receiver_player_name')


In [21]:
qb_stats = pd.DataFrame({
    'Players': ['Lamar Jackson', 'Dak Prescott', 'Brock Purdy', 'Josh Allen'],
    'Total Yards': [4499, 4479, 4424, 4404],
    'Touchdowns - Turnovers': [18, 24, 20, 21],
    'Completion Percentage': [67.2, 68.4, 69.4, 65.6],
    'Team Record': ['13-3', '11-5', '12-4', '10-6']
})

# Assign scores based on stats
qb_stats['Score'] = qb_stats['Total Yards'] + qb_stats['Touchdowns - Turnovers'] + qb_stats['Completion Percentage']

# Calculate total score
total_score = qb_stats['Score'].sum()

# Calculate percentage probability for each player
qb_stats['Probability'] = qb_stats['Score'] / total_score * 100

# Predict MVP
mvp_candidate = qb_stats.loc[qb_stats['Team Record'].idxmax()]

# Display MVP candidate and probabilities
print("MVP Prediction:")
print(mvp_candidate)

print("\nPlayer Probabilities:")
print(qb_stats[['Players', 'Probability']])

MVP Prediction:
Players                   Lamar Jackson
Total Yards                        4499
Touchdowns - Turnovers               18
Completion Percentage              67.2
Team Record                        13-3
Score                            4584.2
Probability                   25.243948
Name: 0, dtype: object

Player Probabilities:
         Players  Probability
0  Lamar Jackson    25.243948
1   Dak Prescott    25.173462
2    Brock Purdy    24.854072
3     Josh Allen    24.728518


In [22]:
offensive_stats = pd.DataFrame({
    'Players': ['Christian McCaffrey', 'Tyreek Hill', 'Ceedee Lamb'],
    'Total Yards': [2023, 1814, 1862],
    'Average Per Touch': [6.05, 11.4, 10.55],
    'Receptions': [67, 119, 135],
    'Touchdowns - Turnovers': [19, 12, 12],
    'Team Record': ['12-5', '11-6', '12-5']
})

# Assign scores based on stats
offensive_stats['Score'] = offensive_stats['Total Yards'] + offensive_stats['Average Per Touch'] + offensive_stats['Receptions'] + offensive_stats['Touchdowns - Turnovers']

# Calculate total score
total_score = offensive_stats['Score'].sum()

# Calculate percentage probability for each player
offensive_stats['Probability'] = offensive_stats['Score'] / total_score * 100

# Predict OPOY
opoy_candidate = offensive_stats.loc[offensive_stats['Team Record'].idxmax()]

# Display OPOY candidate and probabilities
print("OPOY Prediction:")
print(opoy_candidate)

print("\nPlayer Probabilities:")
print(offensive_stats[['Players', 'Probability']])

OPOY Prediction:
Players                   Christian McCaffrey
Total Yards                              2023
Average Per Touch                        6.05
Receptions                                 67
Touchdowns - Turnovers                     19
Team Record                              12-5
Score                                 2115.05
Probability                         34.724183
Name: 0, dtype: object

Player Probabilities:
               Players  Probability
0  Christian McCaffrey    34.724183
1          Tyreek Hill    32.119521
2          Ceedee Lamb    33.156296


In [41]:
defensive_stats = pd.DataFrame({
    'Players': ['Myles Garrett', 'TJ Watt', 'Micah Parsons'],
    'Solo Tackles + Assists': [42, 68, 64],
    'Sacks': [14, 19, 14],
    'Forced Fumbles': [4, 4, 1],
    'Stuffs': [12, 9, 12],
    'Pass-Rush PFF Grades': [94.8, 91.7, 93.7],
    'Team Defense Ranking': [1, 21, 5],
    'Team Record': ['11-6', '10-7', '12-5']
})

# Assign scores based on stats
defensive_stats['Score'] = defensive_stats['Solo Tackles + Assists'] + defensive_stats['Sacks'] + defensive_stats['Forced Fumbles'] + defensive_stats['Stuffs']

# Calculate total score
total_score = defensive_stats['Score'].sum()

# Subtract 'Team Defense Ranking' from 'Score'
defensive_stats['Score'] = defensive_stats['Score'] - defensive_stats['Team Defense Ranking']

# Multiply the result by 'Pass-Rush PFF Grades'
defensive_stats['Score'] = defensive_stats['Score'] * defensive_stats['Pass-Rush PFF Grades']

# Calculate percentage probability for each player
defensive_stats['Probability'] = defensive_stats['Score'] / total_score

# Normalize the 'Probability' column to add up to 100
defensive_stats['Probability'] = defensive_stats['Probability'] / defensive_stats['Probability'].sum() * 100

# Sort the DataFrame by 'Probability' in descending order
sorted_defensive_stats = defensive_stats.sort_values('Probability', ascending=False)

# Predict DPOY
dpoy_candidate = defensive_stats.loc[defensive_stats['Team Record'].idxmax()]

# Display DPOY candidate and probabilities
print("DPOY Prediction:")
print(dpoy_candidate)

# Sort the DataFrame by 'Probability' in descending order
sorted_defensive_stats = defensive_stats.sort_values('Probability', ascending=False)

# Normalize the 'Probability' column to add up to 100
sorted_defensive_stats['Probability'] = sorted_defensive_stats['Probability'] / sorted_defensive_stats['Probability'].sum() * 100

print("\nPlayer Probabilities:")
print(sorted_defensive_stats[['Players', 'Probability']])


DPOY Prediction:
Players                   Micah Parsons
Solo Tackles + Assists               64
Sacks                                14
Forced Fumbles                        1
Stuffs                               12
Pass-Rush PFF Grades               93.7
Team Defense Ranking                  5
Team Record                        12-5
Score                            8058.2
Probability                   36.572824
Name: 2, dtype: object

Player Probabilities:
         Players  Probability
2  Micah Parsons    36.572824
1        TJ Watt    32.878870
0  Myles Garrett    30.548306


In [88]:
import pandas as pd

df = pd.read_csv('nfl_data.csv')
# Load the age data
age_df = pd.read_csv('database.csv')
# Update the ages by adding 3 as csv file is old
age_df['age'] = age_df['age'] + 3.5
# Use a left join to merge the DataFrames
merged_df = pd.merge(df, age_df, how='left', left_on='player_name', right_on='mergename')
# Filter players who are 28 years old or younger
filtered_df = merged_df[merged_df['age'] <= 28]
# Write merged_df to a CSV file
merged_df.to_csv('merged_df.csv', index=False)
# Write filtered_df to a CSV file
filtered_df.to_csv('filtered_df.csv', index=False)
# Calculate a new column 'total_points' as the sum of fantasy_points and fantasy_points_ppr
filtered_df['total_points'] = filtered_df['fantasy_points'] + filtered_df['fantasy_points_ppr']

# Give more weight to RBs and WRs
position_weights = {'RB': 2.0, 'WR': 2.1, 'QB': 1.2, 'TE': 1.0, 'D/ST': 1.0, 'K': 1.0}
filtered_df['position_weight'] = filtered_df['position'].map(position_weights)
filtered_df['weighted_points'] = filtered_df['total_points'] * filtered_df['position_weight']

# Group by player_name and calculate the total points for each player
player_points = filtered_df.groupby('player_display_name')['weighted_points'].sum()

# Sort the players by total_points in descending order and get the top 15
top_15_players = player_points.sort_values(ascending=False).head(15)

# Print the top 15 players
print(top_15_players)

Series([], Name: weighted_points, dtype: float64)
